In [1]:
import jsonlines
import pandas as pd
import numpy as np
import re
import spacy
import warnings 
warnings.filterwarnings('ignore')

nlp = spacy.load('en_core_web_md')

from spacy.pipeline import EntityRuler
ruler = nlp.add_pipe("entity_ruler", name="ruler", before="ner")
ruler.from_disk("jz_skill_patterns.jsonl")

In [2]:
def parse_skills(resume):
    doc = nlp(resume)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    subset = list(set(subset))
    return subset

def skills_to_doc(skills):
    a = re.sub(r'\'', '', skills)
    b = re.sub(r'\[|\]', '',a)
    c = re.split(r"[,]\s*", b)
    lst_of_string =  ' '.join(str(e) for e in c)
    return lst_of_string


def find_resumes(skills, resume, max_matches):
    similarity_score= []
    append_resume = resume
    eg =  ' '.join(str(e) for e in skills)
    for i in range(3677):
        x = skills_to_doc(resume['skills'].loc[i])
        nlp_score = nlp(x).similarity(nlp(eg))
        similarity_score.append(nlp_score)
    append_resume['Similarity'] = similarity_score
    append_resume = append_resume.sort_values(by=['Similarity'], ascending=False)
    append_resume = append_resume.drop(columns=['Resume'])
    sample_row = pd.DataFrame({'ID':'Sample resume', 'Category':'Need to define', 'skills':eg},index =[0])
    appended_resume = pd.concat([sample_row, append_resume])
    return appended_resume.head(max_matches+1)

In [3]:
resume = pd.read_csv ('Extract skills.csv')
resume

,ID,Category,Resume,skills
0,1,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box...","['Collaboration', 'security', 'Box', 'Data Man..."
1,2,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...,"['databases', 'business']"
2,3,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...,"['support', 'first class', 'Business', 'commun..."
3,4,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...,"['Testing', 'database', 'Multimedia', 'Java', ..."
4,5,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert...","['support', 'collaboration', 'Engineering', 'C..."
...,...,...,...,...
3672,99416532,AVIATION,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"['Engineering', 'material', 'Accounting']"
3673,24589765,AVIATION,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","['design', 'support', 'Support', 'finance', 'c..."
3674,31605080,AVIATION,GEEK SQUAD AGENT Professional...,"['IOS', 'support', 'Software', 'Android', 'Win..."
3675,21190805,AVIATION,PROGRAM DIRECTOR / OFFICE MANAGER ...,"['Twitter', 'support', 'design', 'Operating Sy..."


In [4]:
%time
sample = resume['Resume'].loc[696]
print(resume['Category'].loc[696])

Wall time: 0 ns
DIGITAL-MEDIA


In [5]:
%%time
parsed_skills = parse_skills(sample)
print(type(parsed_skills))
print(parsed_skills)

<class 'list'>
['wireless', 'business', 'Commerce', 'HTML', 'marketing', 'CSS', 'Android', 'functional testing', 'iOS', 'ecommerce', 'front end', 'Wordpress', 'design', 'mobile', 'support', 'Marketing', 'forums', 'email marketing']
Wall time: 243 ms


In [6]:
%%time
find_resumes(parsed_skills, resume, 10)

Wall time: 48.3 s


,ID,Category,skills,Similarity
0,Sample resume,Need to define,wireless business Commerce HTML marketing CSS ...,NaN
696,697,DIGITAL-MEDIA,"['design', 'iOS', 'support', 'Commerce', 'emai...",1.000000
3498,22848179,ARTS,"['Twitter', 'design', 'Finance', 'Communicatio...",0.953560
87,88,DESIGNER,"['big data', 'iOS', 'design', 'Content Managem...",0.951886
2485,16893572,DIGITAL-MEDIA,"['Twitter', 'design', 'MARKETING', 'CSS', 'mar...",0.951464
3288,37375999,PUBLIC-RELATIONS,"['chef', 'support', 'design', 'email marketing...",0.948040
2447,27419236,DIGITAL-MEDIA,"['design', 'SQL', 'Box', 'data management', 'A...",0.945291
2523,12085736,DIGITAL-MEDIA,"['Search Engine', 'design', 'SQL', 'material',...",0.940656
1857,12230301,BUSINESS-DEVELOPMENT,"['design', 'MARKETING', 'search engines', 'BUS...",0.940603
1503,64017585,INFORMATION-TECHNOLOGY,"['support', 'design', 'security', 'Computer Sc...",0.940569


In [7]:
%%time
sample_1 =['branding','collateral','competitive analysis','data analytics','ecommerce']
find_resumes(sample_1, resume, 10)

Wall time: 40.3 s


,ID,Category,skills,Similarity
0,Sample resume,Need to define,branding collateral competitive analysis data ...,NaN
2448,13328680,DIGITAL-MEDIA,"['Search Engine', 'Languages', 'Business', 'da...",0.899977
2358,29770086,CONSULTANT,"['Search Engine', 'Hubspot', 'Ecommerce', 'Ope...",0.883262
2480,39166680,DIGITAL-MEDIA,"['SQL', 'Tableau', 'CRM', 'framework', 'Google...",0.877268
2506,20628003,DIGITAL-MEDIA,"['Search Engine', 'support', 'MARKETING', 'mar...",0.868064
2511,80679862,DIGITAL-MEDIA,"['Search Engine', 'design', 'MARKETING', 'New ...",0.866412
2249,12351749,SALES,"['C++', 'support', 'business', 'Business', 'da...",0.862006
2522,16536141,DIGITAL-MEDIA,"['Twitter', 'design', 'MARKETING', 'Telecommun...",0.857801
1828,79759716,BUSINESS-DEVELOPMENT,"['Twitter', 'design', 'Business', 'HubSpot', '...",0.855590
700,701,DIGITAL-MEDIA,"['Twitter', 'amp', 'Buffer', 'Analytics', 'Goo...",0.854119


In [8]:
def university_parser(doc):
    tokenized_doc = nlp(doc)
    ORG_list = [ent.text for ent in tokenized_doc.ents if ent.label_ == "ORG"]
    lst = ["university","University","UNIVERSITY","institude","Institude","INSTITUDE","college","College","COLLEGE"]
    university = []
    for org in ORG_list:
        for word in lst:
            if word in org:
                university.append(org)
    return university

In [12]:
university_parser(resume['Resume'].loc[1982])

['Faulkner UniversityMontgomery']